In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
import time
import datetime

In [2]:
#运行次数
max_steps = 3001
#图片数量
image_num = 3000

DIR = '/Users/xinyu/Documents/JupyterNotebook/jupyter_notebook/tensorflow/炼数成金/'

start_time = time.time()
#载入数据
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

#设置批次的大小
batch_size = 100

#定义会话
sess = tf.Session()

#计算一共有多少批次
n_batch = mnist.train.num_examples // batch_size

#载入图片
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False,name='embedding')

#命名空间
with tf.name_scope('input'):
    #定义两个placehoder
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')  
    
#显示图片
with tf.name_scope('input_reshape'):
    image_shape_input = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input',image_shape_input,10)

#参数概要
def varialbe_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.sqrt(var-mean)))
        tf.summary.scalar('stddev',stddev)#标准差
        tf.summary.scalar('max',tf.reduce_max(var))#最大值
        tf.summary.scalar('min',tf.reduce_min(var))#最小值
        tf.summary.histogram('histogram',var)#直方图

#定义学习率
lr = tf.Variable(0.1,dtype=tf.float32)

with tf.name_scope('layer'):   
    #构建神经网络
    with tf.name_scope('wights1'):
        W1 = tf.Variable(tf.truncated_normal([784,500],stddev=0.1,name = 'W1'))
        varialbe_summaries(W1)
    with tf.name_scope('wights2'):
        W2 = tf.Variable(tf.truncated_normal([500,300],stddev=0.1,name = 'W2'))
        varialbe_summaries(W2)
    with tf.name_scope('wights3'):
        W3 = tf.Variable(tf.truncated_normal([300,10],stddev=0.1,name = 'W3'))
        varialbe_summaries(W3)
    with tf.name_scope('biases1'):
        b1 = tf.Variable(tf.zeros([500],name='b1'))
        varialbe_summaries(b1)
    with tf.name_scope('biases2'):
        b2 = tf.Variable(tf.zeros([300],name='b2'))
        varialbe_summaries(b2)
    with tf.name_scope('biases1'):
        b3 = tf.Variable(tf.zeros([10],name='b3'))
        varialbe_summaries(b3)
    with tf.name_scope('L1'):
        L1 = tf.nn.tanh(tf.matmul(x,W1)+b1)
        varialbe_summaries(L1)
    with tf.name_scope('L2'):
        L2 = tf.nn.tanh(tf.matmul(L1,W2)+b2)
        varialbe_summaries(L2)
    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(tf.matmul(L2,W3)+b3)
        varialbe_summaries(prediction)
    with tf.name_scope('softmax'): 
        prediction = tf.nn.softmax(prediction)

with tf.name_scope('loss'):
    #交叉熵
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)

with tf.name_scope('train'):
    #使用梯度下降法
    train_step = tf.train.GradientDescentOptimizer(lr).minimize(loss)

#初始化变量
sess.run(tf.global_variables_initializer())

with tf.name_scope('accuracy'):
    with tf.name_scope('corrent_prediction'):
        #存储结果（boool型)
        corrent_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    with tf.name_scope('accuracy'):
        #求准确率
        accuracy = tf.reduce_mean(tf.cast(corrent_prediction,tf.float32))#cast 将布尔型转换为浮点型
        tf.summary.scalar('accuracy',accuracy)

file_dir = DIR+'projector/projector' + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S/')
tf.gfile.MkDir(file_dir)
with open(file_dir+'metadata.tsv','w') as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:],1))
    for i in range(image_num):
        f.write(str(labels[i])+'\n')
        
#合并所有的summary
merged = tf.summary.merge_all()

projector_writer = tf.summary.FileWriter(file_dir,sess.graph)
saver = tf.train.Saver()
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = file_dir+'metadata.tsv'
embed.sprite.image_path = DIR+'projector/data/mnist_10k_sprite.png'
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer,config)

# for i in range(max_steps):
#     #每个批次100个样本
#     batch_xs,batch_ys = mnist.train.next_batch(100)
#     run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
#     run_metadata = tf.RunMetadata()
#     summary, _ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
#     projector_writer.add_run_metadata(run_metadata,'step%03d' % i)
#     projector_writer.add_summary(summary,i)
    
#     if i % 100 == 0:
#         acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
#         print('Iter ' + str(i) + ', Testing Accuracy= ' + str(acc))
        
# saver.save(sess,file_dir + 'a_model.ckpt',global_step=max_steps)
# projector_writer.close()
# sess.close()

# with tf.Session() as sess:
#     sess.run(init)
for i in range(max_steps):
#     sess.run(tf.assign(lr,lr*(0.95**i)))
    batch_xs,batch_ys = mnist.train.next_batch(100)
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary, _ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata,'step%03d' % i)
    projector_writer.add_summary(summary,i)

    acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
    print("Iter" + str(i)+ ":" + "Testing Accuracy" +":"+ str(acc))
        
#         if i % 100 == 0:
#         acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
#         print('Iter ' + str(i) + ', Testing Accuracy= ' + str(acc))
        
saver.save(sess,file_dir + 'a_model.ckpt',global_step=max_steps)
projector_writer.close()
        
end_time = time.time()

print("消耗的时间为：" + str(end_time - start_time))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Iter0:Testing Accuracy:0.1119
Iter1:Testing Accuracy:0.1156
Iter2:Testing Accuracy:0.1182
Iter3:Testing Accura

Iter184:Testing Accuracy:0.2933
Iter185:Testing Accuracy:0.2934
Iter186:Testing Accuracy:0.2936
Iter187:Testing Accuracy:0.2939
Iter188:Testing Accuracy:0.2941
Iter189:Testing Accuracy:0.2946
Iter190:Testing Accuracy:0.2946
Iter191:Testing Accuracy:0.2947
Iter192:Testing Accuracy:0.2947
Iter193:Testing Accuracy:0.295
Iter194:Testing Accuracy:0.2946
Iter195:Testing Accuracy:0.2952
Iter196:Testing Accuracy:0.2958
Iter197:Testing Accuracy:0.2958
Iter198:Testing Accuracy:0.2969
Iter199:Testing Accuracy:0.297
Iter200:Testing Accuracy:0.298
Iter201:Testing Accuracy:0.2983
Iter202:Testing Accuracy:0.2985
Iter203:Testing Accuracy:0.2985
Iter204:Testing Accuracy:0.2996
Iter205:Testing Accuracy:0.3003
Iter206:Testing Accuracy:0.3013
Iter207:Testing Accuracy:0.3019
Iter208:Testing Accuracy:0.3023
Iter209:Testing Accuracy:0.3029
Iter210:Testing Accuracy:0.3037
Iter211:Testing Accuracy:0.3042
Iter212:Testing Accuracy:0.3051
Iter213:Testing Accuracy:0.306
Iter214:Testing Accuracy:0.3079
Iter215:Test

Iter443:Testing Accuracy:0.6305
Iter444:Testing Accuracy:0.6313
Iter445:Testing Accuracy:0.6316
Iter446:Testing Accuracy:0.6319
Iter447:Testing Accuracy:0.6315
Iter448:Testing Accuracy:0.6315
Iter449:Testing Accuracy:0.6325
Iter450:Testing Accuracy:0.6334
Iter451:Testing Accuracy:0.6328
Iter452:Testing Accuracy:0.6329
Iter453:Testing Accuracy:0.6331
Iter454:Testing Accuracy:0.6341
Iter455:Testing Accuracy:0.6354
Iter456:Testing Accuracy:0.6349
Iter457:Testing Accuracy:0.6354
Iter458:Testing Accuracy:0.6379
Iter459:Testing Accuracy:0.6394
Iter460:Testing Accuracy:0.6406
Iter461:Testing Accuracy:0.6416
Iter462:Testing Accuracy:0.6414
Iter463:Testing Accuracy:0.6428
Iter464:Testing Accuracy:0.6425
Iter465:Testing Accuracy:0.643
Iter466:Testing Accuracy:0.6436
Iter467:Testing Accuracy:0.6435
Iter468:Testing Accuracy:0.6439
Iter469:Testing Accuracy:0.6447
Iter470:Testing Accuracy:0.6457
Iter471:Testing Accuracy:0.646
Iter472:Testing Accuracy:0.6459
Iter473:Testing Accuracy:0.6474
Iter474:Te

Iter700:Testing Accuracy:0.6979
Iter701:Testing Accuracy:0.6984
Iter702:Testing Accuracy:0.6982
Iter703:Testing Accuracy:0.6986
Iter704:Testing Accuracy:0.6983
Iter705:Testing Accuracy:0.6992
Iter706:Testing Accuracy:0.6986
Iter707:Testing Accuracy:0.6986
Iter708:Testing Accuracy:0.6993
Iter709:Testing Accuracy:0.6994
Iter710:Testing Accuracy:0.6995
Iter711:Testing Accuracy:0.6998
Iter712:Testing Accuracy:0.6996
Iter713:Testing Accuracy:0.7
Iter714:Testing Accuracy:0.7003
Iter715:Testing Accuracy:0.7001
Iter716:Testing Accuracy:0.7006
Iter717:Testing Accuracy:0.7005
Iter718:Testing Accuracy:0.7005
Iter719:Testing Accuracy:0.7004
Iter720:Testing Accuracy:0.6997
Iter721:Testing Accuracy:0.7001
Iter722:Testing Accuracy:0.7007
Iter723:Testing Accuracy:0.7007
Iter724:Testing Accuracy:0.7007
Iter725:Testing Accuracy:0.701
Iter726:Testing Accuracy:0.7014
Iter727:Testing Accuracy:0.7017
Iter728:Testing Accuracy:0.7016
Iter729:Testing Accuracy:0.7028
Iter730:Testing Accuracy:0.7016
Iter731:Test

Iter958:Testing Accuracy:0.7207
Iter959:Testing Accuracy:0.7207
Iter960:Testing Accuracy:0.7211
Iter961:Testing Accuracy:0.7213
Iter962:Testing Accuracy:0.7209
Iter963:Testing Accuracy:0.7209
Iter964:Testing Accuracy:0.7205
Iter965:Testing Accuracy:0.7206
Iter966:Testing Accuracy:0.7206
Iter967:Testing Accuracy:0.7207
Iter968:Testing Accuracy:0.7207
Iter969:Testing Accuracy:0.7212
Iter970:Testing Accuracy:0.7217
Iter971:Testing Accuracy:0.7218
Iter972:Testing Accuracy:0.7211
Iter973:Testing Accuracy:0.7216
Iter974:Testing Accuracy:0.7218
Iter975:Testing Accuracy:0.7215
Iter976:Testing Accuracy:0.7216
Iter977:Testing Accuracy:0.722
Iter978:Testing Accuracy:0.7222
Iter979:Testing Accuracy:0.7221
Iter980:Testing Accuracy:0.7223
Iter981:Testing Accuracy:0.7224
Iter982:Testing Accuracy:0.7223
Iter983:Testing Accuracy:0.7223
Iter984:Testing Accuracy:0.7222
Iter985:Testing Accuracy:0.7222
Iter986:Testing Accuracy:0.722
Iter987:Testing Accuracy:0.7225
Iter988:Testing Accuracy:0.7222
Iter989:Te

...

Iter1962:Testing Accuracy:0.7442
Iter1963:Testing Accuracy:0.7442
Iter1964:Testing Accuracy:0.7444
Iter1965:Testing Accuracy:0.7444
Iter1966:Testing Accuracy:0.7441
Iter1967:Testing Accuracy:0.744
Iter1968:Testing Accuracy:0.7441
Iter1969:Testing Accuracy:0.7439
Iter1970:Testing Accuracy:0.7442
Iter1971:Testing Accuracy:0.7437
Iter1972:Testing Accuracy:0.7436
Iter1973:Testing Accuracy:0.7439
Iter1974:Testing Accuracy:0.7439
Iter1975:Testing Accuracy:0.7437
Iter1976:Testing Accuracy:0.7439
Iter1977:Testing Accuracy:0.744
Iter1978:Testing Accuracy:0.7438
Iter1979:Testing Accuracy:0.7439
Iter1980:Testing Accuracy:0.744
Iter1981:Testing Accuracy:0.744
Iter1982:Testing Accuracy:0.744
Iter1983:Testing Accuracy:0.7442
Iter1984:Testing Accuracy:0.744
Iter1985:Testing Accuracy:0.7441
Iter1986:Testing Accuracy:0.7444
Iter1987:Testing Accuracy:0.7442
Iter1988:Testing Accuracy:0.7439
Iter1989:Testing Accuracy:0.7443
Iter1990:Testing Accuracy:0.7444
Iter1991:Testing Accuracy:0.7442
Iter1992:Testing

Iter2212:Testing Accuracy:0.7467
Iter2213:Testing Accuracy:0.7468
Iter2214:Testing Accuracy:0.7467
Iter2215:Testing Accuracy:0.7467
Iter2216:Testing Accuracy:0.7465
Iter2217:Testing Accuracy:0.7467
Iter2218:Testing Accuracy:0.7467
Iter2219:Testing Accuracy:0.7468
Iter2220:Testing Accuracy:0.7467
Iter2221:Testing Accuracy:0.7468
Iter2222:Testing Accuracy:0.7468
Iter2223:Testing Accuracy:0.747
Iter2224:Testing Accuracy:0.7467
Iter2225:Testing Accuracy:0.7472
Iter2226:Testing Accuracy:0.7474
Iter2227:Testing Accuracy:0.747
Iter2228:Testing Accuracy:0.7473
Iter2229:Testing Accuracy:0.7476
Iter2230:Testing Accuracy:0.7475
Iter2231:Testing Accuracy:0.748
Iter2232:Testing Accuracy:0.7477
Iter2233:Testing Accuracy:0.7479
Iter2234:Testing Accuracy:0.7477
Iter2235:Testing Accuracy:0.7479
Iter2236:Testing Accuracy:0.7477
Iter2237:Testing Accuracy:0.7477
Iter2238:Testing Accuracy:0.7477
Iter2239:Testing Accuracy:0.7479
Iter2240:Testing Accuracy:0.7477
Iter2241:Testing Accuracy:0.7476
Iter2242:Test

Iter2464:Testing Accuracy:0.7585
Iter2465:Testing Accuracy:0.7585
Iter2466:Testing Accuracy:0.7589
Iter2467:Testing Accuracy:0.759
Iter2468:Testing Accuracy:0.7592
Iter2469:Testing Accuracy:0.7592
Iter2470:Testing Accuracy:0.7591
Iter2471:Testing Accuracy:0.7596
Iter2472:Testing Accuracy:0.7597
Iter2473:Testing Accuracy:0.7596
Iter2474:Testing Accuracy:0.7596
Iter2475:Testing Accuracy:0.7596
Iter2476:Testing Accuracy:0.7597
Iter2477:Testing Accuracy:0.7599
Iter2478:Testing Accuracy:0.7598
Iter2479:Testing Accuracy:0.7595
Iter2480:Testing Accuracy:0.7599
Iter2481:Testing Accuracy:0.7598
Iter2482:Testing Accuracy:0.76
Iter2483:Testing Accuracy:0.7598
Iter2484:Testing Accuracy:0.7608
Iter2485:Testing Accuracy:0.7609
Iter2486:Testing Accuracy:0.7618
Iter2487:Testing Accuracy:0.7618
Iter2488:Testing Accuracy:0.7615
Iter2489:Testing Accuracy:0.7616
Iter2490:Testing Accuracy:0.7617
Iter2491:Testing Accuracy:0.7616
Iter2492:Testing Accuracy:0.7616
Iter2493:Testing Accuracy:0.7616
Iter2494:Test

Iter2713:Testing Accuracy:0.7928
Iter2714:Testing Accuracy:0.7936
Iter2715:Testing Accuracy:0.7943
Iter2716:Testing Accuracy:0.7942
Iter2717:Testing Accuracy:0.7942
Iter2718:Testing Accuracy:0.7952
Iter2719:Testing Accuracy:0.7954
Iter2720:Testing Accuracy:0.797
Iter2721:Testing Accuracy:0.7976
Iter2722:Testing Accuracy:0.7983
Iter2723:Testing Accuracy:0.7983
Iter2724:Testing Accuracy:0.7996
Iter2725:Testing Accuracy:0.8003
Iter2726:Testing Accuracy:0.8008
Iter2727:Testing Accuracy:0.8011
Iter2728:Testing Accuracy:0.8017
Iter2729:Testing Accuracy:0.8026
Iter2730:Testing Accuracy:0.803
Iter2731:Testing Accuracy:0.8037
Iter2732:Testing Accuracy:0.8043
Iter2733:Testing Accuracy:0.8048
Iter2734:Testing Accuracy:0.8054
Iter2735:Testing Accuracy:0.8063
Iter2736:Testing Accuracy:0.8071
Iter2737:Testing Accuracy:0.8086
Iter2738:Testing Accuracy:0.8091
Iter2739:Testing Accuracy:0.8095
Iter2740:Testing Accuracy:0.8101
Iter2741:Testing Accuracy:0.8108
Iter2742:Testing Accuracy:0.8112
Iter2743:Tes

Iter2963:Testing Accuracy:0.8682
Iter2964:Testing Accuracy:0.8686
Iter2965:Testing Accuracy:0.8687
Iter2966:Testing Accuracy:0.8688
Iter2967:Testing Accuracy:0.8688
Iter2968:Testing Accuracy:0.8687
Iter2969:Testing Accuracy:0.8685
Iter2970:Testing Accuracy:0.8689
Iter2971:Testing Accuracy:0.8694
Iter2972:Testing Accuracy:0.8695
Iter2973:Testing Accuracy:0.869
Iter2974:Testing Accuracy:0.8694
Iter2975:Testing Accuracy:0.8695
Iter2976:Testing Accuracy:0.8695
Iter2977:Testing Accuracy:0.8691
Iter2978:Testing Accuracy:0.8694
Iter2979:Testing Accuracy:0.8692
Iter2980:Testing Accuracy:0.8695
Iter2981:Testing Accuracy:0.8694
Iter2982:Testing Accuracy:0.8696
Iter2983:Testing Accuracy:0.8693
Iter2984:Testing Accuracy:0.8691
Iter2985:Testing Accuracy:0.869
Iter2986:Testing Accuracy:0.8688
Iter2987:Testing Accuracy:0.8688
Iter2988:Testing Accuracy:0.8688
Iter2989:Testing Accuracy:0.8689
Iter2990:Testing Accuracy:0.8688
Iter2991:Testing Accuracy:0.8691
Iter2992:Testing Accuracy:0.87
Iter2993:Testi